# Añadir variables

De momento, el dataframe con las semillas y los resultados no tiene el valor de HP o HP inverso para los diferentes valores de curva y b. Aquí vamos a cargar ese .csv, añadir estas columnas y a guardar una copia

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
from src.aitor_utils import Graph_gen, C_from_gr, get_blockade_configurations2, Gurobi_solver, Gurobi_to_gr, gaussian


df = pd.read_csv('../Comparacion drivings optimos/simul_drivings_opt_1000_seeds.csv')
df.head()

,seed,AR_max,reached_0.95,reached_0.98,reached_0.99,reached_0.999,hist_AR
0,628,0.975928,5,NaN,NaN,NaN,"[np.float64(0.8476024522292993), np.float64(0...."
1,629,0.999910,5,5.0,5.0,5.0,"[np.float64(0.8733235022650057), np.float64(0...."
2,630,0.999872,5,5.0,5.0,5.0,"[np.float64(0.9208468236714975), np.float64(0...."
3,631,0.999183,5,5.0,5.0,268.0,"[np.float64(0.8699287773666359), np.float64(0...."
4,632,0.999794,1,1.0,1.0,5.0,"[np.float64(0.9961055080721747), np.float64(0...."


In [9]:
curve_list = ['linear', 'sqrt', 'square', 'cubic', 'cubic_root']
b_list = np.linspace(0.1, 1, 30)**3
scale = 5e-6
clasif = pd.DataFrame(columns=['seed', 'curva', 'b', 'hp', 'hp_inv'])

for i in range(len(df)):
    seed = df['seed'][i]
    n_nodos = df['n_nodos'][i]
    atoms_w = df['atoms_w'][i]
    atoms_l = df['atoms_l'][i]
    np.random.seed(seed)
    G, register = Graph_gen(n_nodos, atoms_w, atoms_l, scale)

    # Guardamos propiedades relevantes del grafo
    og_weights = np.array(list(nx.get_node_attributes(G, 'weight').values()))
    positions = np.array(list(nx.get_node_attributes(G, 'pos').values()))
    
    # Calcular el espectro del grafo
    conf = get_blockade_configurations2(positions, 7.2e-6)
    espectro = pd.DataFrame(conf, columns=['bitstring'])
    espectro['energia'] = espectro['bitstring'].apply(lambda x: C_from_gr(x, og_weights, G.nodes()))
    espectro = espectro.sort_values('energia', ascending=True)

    e_min = espectro['energia'].values[0]

    for c in curve_list:
        for b in b_list:
            espectro['ponderacion'] = (espectro['energia'].apply(lambda x: gaussian(E=x, a=e_min, b=b, met=c)))
            hp = sum(espectro['ponderacion'].values)
            hp_inv = 1/hp
            new_data = {'seed': seed, 'curva': c, 'b': b, 'hp': hp, 'hp_inv': hp_inv}
            clasif = pd.concat([clasif, pd.DataFrame([new_data])], ignore_index=True)

C:\Users\aitoo\AppData\Local\Temp\ipykernel_16524\1249082179.py:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  clasif = pd.concat([clasif, pd.DataFrame([new_data])], ignore_index=True)


In [13]:
df2 = df.merge(clasif, on=['seed'], how='left')

In [14]:
df2.to_csv('drivings_opt_1000_seeds_hp.csv', index=False)

In [7]:
len(df)

150000

In [12]:
df_2 = pd.read_csv('../Estudio opt por grafo/opt_por_seed.csv')
df_2 = df_2[['seed', 'AR_max']]
df = pd.read_csv('drivings_opt_1000_seeds_hp.csv')
df = df.merge(df_2, on=['seed'], how='left', suffixes=('', '_y'))


In [13]:
df.head()

,n_nodos,atoms_w,atoms_l,seed,p_0,p_1,e_gap,e_0,e_1,succ,ar,curva,b,hp,hp_inv,AR_max
0,7,4,3,0,0.9244,0.0756,0.071,-1.9603,-1.8893,0.9244,0.997262,linear,0.001000,1.0,1.0,0.999789
1,7,4,3,0,0.9244,0.0756,0.071,-1.9603,-1.8893,0.9244,0.997262,linear,0.002250,1.0,1.0,0.999789
2,7,4,3,0,0.9244,0.0756,0.071,-1.9603,-1.8893,0.9244,0.997262,linear,0.004257,1.0,1.0,0.999789
3,7,4,3,0,0.9244,0.0756,0.071,-1.9603,-1.8893,0.9244,0.997262,linear,0.007201,1.0,1.0,0.999789
4,7,4,3,0,0.9244,0.0756,0.071,-1.9603,-1.8893,0.9244,0.997262,linear,0.011260,1.0,1.0,0.999789


In [14]:
df.to_csv('drivings_opt_1000_seeds_hp.csv', index=False)